# Training an object detection AI model using YOLOv5 with OpenShift Data Science (Part 3)

### Setting up the environment and installing suitable dependencies

In [1]:
!pip install -q numpy onnx onnxruntime opencv_python_headless seaborn torch torchvision


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# Part 3: Convert the trained to ONNX format, and the upload to S3 bucket for serving

### Export the model to ONNX
- Eventually modify the image size if you selected a model with the "6" suffix, as image size is 1280 (and not 640)
- Stay at opset 16 for the moment for OpenVino model server compatibility

In [2]:
!python export.py --weights yolov5n.pt --include onnx --imgsz 640 --opset 16

export: data=data/coco128.yaml, weights=['yolov5n.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=16, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 2023-11-7 Python-3.9.16 torch-1.13.1+cu117 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients

PyTorch: starting from yolov5n.pt with output shape (1, 25200, 85) (3.9 MB)

ONNX: starting export with onnx 1.15.0...
ONNX: export success ✅ 0.8s, saved as yolov5n.onnx (7.6 MB)

Export complete (1.2s)
Results saved to /opt/app-root/src/yolov5-object-detection-end-to-end-demo/workbench-demo
Detect:          python detect.py --weights yolov5n.onnx 
Validate:        python val.py --weights yolov5n.onnx 
PyTorch Hub:     model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5n.onnx')  
Visualize:       htt

### Save the ONNX model to where your OpenShift Data Science project data connection points to (which is an S3 bucket)

In [3]:
import helper_functions as helper
import os
from datetime import datetime

In [4]:
now = datetime.now() # Get current datetime
date_string = now.strftime("%Y-%m-%d-UTC%H-%M-%S") # Convert datetime object to string
print(date_string)

2023-11-07-UTC12-51-26


In [5]:
helper.upload_to_s3(
    endpoint_url=os.environ['AWS_S3_ENDPOINT'],
    access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    secret_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    file_name='./yolov5n.onnx',
    bucket=os.environ['AWS_S3_BUCKET'],
    object_name='yolov5n-' + date_string + '.onnx')

File uploaded successfully


True